In [4]:
import pandas as pd
import polars as pl
from utils.util import generate_postgres_ddl, df_to_db, read_database, read_from_sqlite, engine
from config import sheet_config_travel_journey
from openpyxl.utils import column_index_from_string
import openpyxl
import time
import datetime
import os
from pyxlsb import open_workbook
import sqlite3

In [12]:
query = """
    select
        kode_lokasi,
        sheet,
        FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)) as start_rentang_survei,
        case
            when FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)) < 6 then FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)) + 25
            else FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float))
        end as rank_rentang_survei,
        (FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)):: varchar) || '.00 - ' || (CEIL(cast(trim(split_part(rentang_survei, '-', 2)) as float)):: varchar) || '.00' as rentang_survei_,
        sum("Gol-6") as gol_6,
        sum("Gol-1-a") as gol_1_a,
        sum("Gol-1-b") as gol_1_b,
        sum("Gol-1-c") as gol_1_c,
        sum("Gol-1-d") as gol_1_d,
        sum("Gol-1-a") as gol_1_e,
        sum("Gol-2") as gol_2,
        sum("Gol-3") as gol_3,
        sum("Gol-4") as gol_4,
        sum("Gol-5") as gol_5,
        sum(total_tanpa_sepeda_motor) as total_tanpa_sepeda_motor,
        sum(total_dengan_sepeda_motor) as total_dengan_sepeda_motor
    from
        survey_traffic_counting
    group by
        kode_lokasi,
        sheet,
        start_rentang_survei,
        rank_rentang_survei,
        rentang_survei_
    order by sheet, rank_rentang_survei
    ;
"""
df = read_database(engine=engine, query=query)

# Export the DataFrame to a pickle file
pickle_file_path = 'fetch_stats_data.pkl'
df.to_pickle(pickle_file_path)
print(f"\nDataFrame exported to {pickle_file_path}")

df


DataFrame exported to fetch_stats_data.pkl


,kode_lokasi,sheet,start_rentang_survei,rank_rentang_survei,rentang_survei_,gol_6,gol_1_a,gol_1_b,gol_1_c,gol_1_d,gol_1_e,gol_2,gol_3,gol_4,gol_5,total_tanpa_sepeda_motor,total_dengan_sepeda_motor
0,ADDBDG-1,HK_Arah-1,6.0,6.0,6.00 - 7.00,453,6,0,0,0,6,1,0,0,0,15,468
1,ADDJBO - 3,HK_Arah-1,6.0,6.0,6.00 - 7.00,2164,1297,0,5,0,1297,4,0,0,0,1311,3475
2,ADDJBO-1,HK_Arah-1,6.0,6.0,6.00 - 7.00,1437,128,26,2,0,128,10,2,6,2,187,1624
3,ADDJBO-13,HK_Arah-1,6.0,6.0,6.00 - 7.00,1578,1095,1,0,0,1095,18,0,0,0,1132,2710
4,ADDJBO-15,HK_Arah-1,6.0,6.0,6.00 - 7.00,1694,192,2,1,0,192,31,0,0,0,239,1933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22075,PLUSJAWA-5,HL_Arah-2,5.0,30.0,5.00 - 6.00,1093,397,6,12,3,397,11,4,0,0,460,1553
22076,PLUSJAWA-6,HL_Arah-2,5.0,30.0,5.00 - 6.00,1630,414,16,49,17,414,168,73,21,22,912,2542
22077,PLUSJAWA-7,HL_Arah-2,5.0,30.0,5.00 - 6.00,722,173,13,3,5,173,89,65,3,15,412,1134
22078,PLUSJAWA-8,HL_Arah-2,5.0,30.0,5.00 - 6.00,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
survey_asal_tujuan_xlsx = "/mnt/d/PROJ UI/Data Primer Test/Data Primer Test/Survei Asal Tujuan/RSI JM Jawa 2024 - WorkFile - v511.xlsx"
survey_traffic_counting_jabo_xlsx = "/mnt/d/PROJ UI/Data Primer Test/Data Primer Test/Survei Traffic Counting/Rekapitulasi/Database Traffic Counting_JABO.xlsx"
survey_travel_journey_xlsx = "/mnt/d/PROJ UI/Data Primer Test/Data Primer Test/Survei Travel Journey/Travel Journeyy.xlsx"

# Survei Asal Tujuan

In [3]:
wb = openpyxl.load_workbook(survey_asal_tujuan_xlsx, data_only=True)
sheet_name = wb["FixUse"]  # Specify the sheet name

columns = [
    'kode_titik',
    'arah',
    'jenis',
    'trayek',
    'penumpang',
    'asal_tempat',
    'asal_tanggal',
    'asal_waktu',
    'asal_kab_kota',
    'asal_kec',
    'asal_kel',
    'penggunaan_tol',
    'tujuan_tempat',
    'tujuan_tanggal',
    'tujuan_waktu',
    'tujuan_kab_kota',
    'tujuan_kec',
    'tujuan_kel',
    'kode'
]

In [ ]:
data = []
for row in sheet_name.iter_rows(values_only=False, min_row=3):
    d_dict = {}
    if all(cell.value is None for cell in row):
        break
    
    for i in range(0, len(columns)):
        if row[i].is_date:
            # print(row[i].value, row[i].number_format)
            d_dict[columns[i]] = row[i].number_format if row[i].value is None else str(row[i].value)
        else:
            d_dict[columns[i]]= row[i].value if row[i].value is not None else None
    data.append(d_dict)

df_survey_asal_tujuan = pl.DataFrame(data)
# print(df_survey_asal_tujuan)

ddl_survey_asal_tujuan = generate_postgres_ddl(df=df_survey_asal_tujuan, table_name="survey_asal_tujuan")
print(ddl_survey_asal_tujuan)
df_to_db(df=df_survey_asal_tujuan, table_name="survey_asal_tujuan")

# Survei Traffic Counting

In [6]:
wb = openpyxl.load_workbook(survey_traffic_counting_jabo_xlsx, data_only=True)  # data_only=False keeps the formulas
sheet_name = wb["Data TC"]  # Specify the sheet name

columns = [
    'tc_code',
    'latitude',
    'longitude',
    'tahun',
    'bulan',
    'tanggal',
    'hari_kerja_hari_libur',
    'arah',
    'dari',
    'ke',
    'durasi',
    'waktu',
    'golongan_1',
    'golongan_2',
    'golongan_3',
    'golongan_4',
    'golongan_5',
    'golongan_6',
    'golongan_7',
    'golongan_8',
    'golongan_9',
    'golongan_10',
    'golongan_11',
    'golongan_12',
    'keterangan',
]

In [8]:
data = []
for row in sheet_name.iter_rows(values_only=True):
    d_dict = {}
    if all(cell is None for cell in row):
        break
    for i in range(0, len(columns)):
        d_dict[columns[i]] = row[i]
    data.append(d_dict)

In [ ]:
df_survey_traffic_counting_jabo = pl.DataFrame(data)
df_survey_traffic_counting_jabo = df_survey_traffic_counting_jabo.slice(1)

print(df_survey_traffic_counting_jabo)
ddl_survey_traffic_counting_jabo = generate_postgres_ddl(df=df_survey_traffic_counting_jabo, table_name="survey_traffic_counting_jabo")
print(ddl_survey_traffic_counting_jabo)
df_to_db(df=df_survey_traffic_counting_jabo, table_name="survey_traffic_counting_jabo")

# Survei Traffic Counting 2

In [72]:
survey_traffic_counting_folders = [
    "/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/BDG",
    "/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/JABO",
    "/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/JAWA"
    ]

In [73]:
config_data = [
    [4, 2, "hari_tanggal"],
    [4, 12, "cuaca"],
    [5, 2, "nama_ruas_jalan"],
    [5, 8, "koordinat_lokasi"],
    [5, 12, "arah_dari"],
    [6, 2, "surveyor_rekam_hitung"],
    [6, 8, "durasi"],
    [6, 12, "arah_menuju"],
    [7, 2, "kode_lokasi"],
    [7, 12, "kode_arah"],
]

config_data_profil = [
    [6, 5, "catatan"],
    [3, 9, "jam_puncak_arah_1"],
    [4, 9, "vol_jam_puncak_arah_1"],
    [6, 9, "jam_puncak_arah_2"],
    [7, 9, "vol_jam_puncak_arah_2"],
]

table_data = {
    "i": [12, 203],
    "c": [1, 13],
    "mapping_col": {
        1: "rentang_survei",
        2: "Gol-6",
        3: "Gol-1-a",
        4: "Gol-1-b",
        5: "Gol-1-c",
        6: "Gol-1-d",
        7: "Gol-1-e",
        8: "Gol-2",
        9: "Gol-3",
        10: "Gol-4",
        11: "Gol-5",
        12: "total_tanpa_sepeda_motor",
        13: "total_dengan_sepeda_motor",
    }
}

data = []
for folder in survey_traffic_counting_folders:
    for filename in os.listdir(folder):
        f = os.path.join(folder, filename)
        print(filename)
        
        with open_workbook(f) as wb:
            profil_dict = {}
            for sheet_name in wb.sheets:
                if str(sheet_name).startswith(("Profil")):
                    with wb.get_sheet(sheet_name) as sheet:
                        for i, row in enumerate(sheet):
                            for c in range(len(row)):
                                for config in config_data_profil:
                                    if i == config[0] and c == config[1]:
                                        if str(sheet_name).endswith(("HK")):
                                            print(i, c, config[2])
                                            profil_dict[f"{config[2]}_hk"] = "%.2f" % row[c].v if isinstance(row[c].v, float) else row[c].v
                                        else:
                                            profil_dict[f"{config[2]}_hl"] = "%.2f" % row[c].v if isinstance(row[c].v, float) else row[c].v
                    
            for sheet_name in wb.sheets:
                if str(sheet_name).startswith(("HK_Arah", "HL_Arah")):
                    with wb.get_sheet(sheet_name) as sheet:
                        base_dict = {}
                        base_dict["filename"] = filename
                        base_dict["region"] = folder.split('/')[-1]
                        base_dict["sheet"] = sheet_name
                        for i, row in enumerate(sheet):
                            for c in range(len(row)):
                                if row[c].v:
                                    for config in config_data:
                                        if i == config[0] and c == config[1]:
                                            base_dict[config[2]] = int(row[c].v) if isinstance(row[c].v, float) else row[c].v
                        
                        d_dict = {}
                        for i, row in enumerate(sheet):
                            for c in range(len(row)):
                                if i >= table_data["i"][0] and i <= table_data["i"][1] and c >= table_data["c"][0] and c <= table_data["c"][1]:
                                    if c == 1:
                                        d_dict[table_data["mapping_col"][c]] = row[c].v if row[c].v else None
                                        # print(table_data["mapping_col"][c], row[c].v if row[c].v else None)
                                    else:
                                        d_dict[table_data["mapping_col"][c]] = int(row[c].v) if isinstance(row[c].v, float) else 0 
                                        # print(table_data["mapping_col"][c], int(row[c].v) if row[c].v else None)
                            if d_dict:
                                complete_dict = {**base_dict, **profil_dict, **d_dict}
                                data.append(complete_dict)
            print(len(data))
# df = pl.DataFrame(data)
# df["rentang_survei"].unique()
# df
            
# ddl = generate_postgres_ddl(df=df, table_name="survey_traffic_counting")    
# print(ddl)

ADDBDG-1.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
1200
BDG-1.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
2400
BDG-12.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
3600
BDG-14.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
4800
BDG-15.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
6000
BDG-16.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
7200
BDG-17.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
8400
BDG-18.xlsb
3 9 jam_puncak_arah_1
4 9 vol_jam_puncak_arah_1
6 5 catatan
6 9 jam_puncak_arah_2
7 9 vol_jam_puncak_arah_2
9600

In [77]:
df = pl.DataFrame(data)
# df
# ddl = generate_postgres_ddl(df=df, table_name="survey_traffic_counting")    
# print(ddl)

df_to_db(df=df, table_name="survey_traffic_counting")

In [62]:

with open_workbook("/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/BDG/ADDBDG-1.xlsb") as wb:
    for sheet_name in wb.sheets:
        if str(sheet_name).startswith(("Profil")):
            with wb.get_sheet("Profil_HK") as sheet:
                target_row = 5
                target_col = column_index_from_string("C")
                for i, row in enumerate(sheet):
                    row_data = []
                    for c in range(len(row)):
                        if row[c].v:
                            row_data.append({
                                "v": row[c].v,
                                "i": i,
                                "c": c
                                
                            })
                    print(row_data)

[{'v': 'ADDBDG-1_HK.xlsx', 'i': 0, 'c': 0}]
[]
[{'v': 'No./Kode Lokasi :', 'i': 2, 'c': 1}, {'v': 'ADDBDG-1', 'i': 2, 'c': 2}, {'v': 'Jumlah arah :', 'i': 2, 'c': 4}, {'v': 2.0, 'i': 2, 'c': 5}, {'v': 'Arah-1', 'i': 2, 'c': 8}]
[{'v': 'Durasi :', 'i': 3, 'c': 1}, {'v': '16', 'i': 3, 'c': 2}, {'v': 'jam', 'i': 3, 'c': 3}, {'v': 'Jumlah lajur per arah:', 'i': 3, 'c': 4}, {'v': '1', 'i': 3, 'c': 5}, {'v': 'Jam puncak :', 'i': 3, 'c': 8}, {'v': '17.00 - 18.00', 'i': 3, 'c': 9}, {'v': 'Ekivalen mobil penumpang', 'i': 3, 'c': 24}]
[{'v': 'Koordinat Lokasi :', 'i': 4, 'c': 1}, {'v': '-7.00150683, 107.7208633', 'i': 4, 'c': 2}, {'v': 'Terbagi (D) / tidak terbagi (UD) :', 'i': 4, 'c': 4}, {'v': 'UD', 'i': 4, 'c': 5}, {'v': 'Vol. jam puncak (smp/jam):', 'i': 4, 'c': 8}, {'v': 518.5, 'i': 4, 'c': 9}, {'v': 'MCY', 'i': 4, 'c': 24}, {'v': 'CAR', 'i': 4, 'c': 25}, {'v': 'LGV', 'i': 4, 'c': 26}, {'v': 'MGV', 'i': 4, 'c': 27}, {'v': 'HGV', 'i': 4, 'c': 28}, {'v': 'Angkot', 'i': 4, 'c': 29}, {'v': 'Bus

In [ ]:
config_data = [
    [4, 2, "hari_tanggal"],
    [4, 12, "cuaca"],
    [5, 2, "nama_ruas_jalan"],
    [5, 8, "koordinat_lokasi"],
    [5, 12, "arah_dari"],
    [6, 2, "surveyor_rekam_hitung"],
    [6, 8, "durasi"],
    [6, 12, "arah_menuju"],
    [7, 2, "kode_lokasi"],
    [7, 12, "kode_arah"],
]

table_data = {
    "i": [12, 203],
    "c": [1, 13],
    "mapping_col": {
        1: "rentang_survei",
        2: "Gol-6",
        3: "Gol-1-a",
        4: "Gol-1-b",
        5: "Gol-1-c",
        6: "Gol-1-d",
        7: "Gol-1-e",
        8: "Gol-2",
        9: "Gol-3",
        10: "Gol-4",
        11: "Gol-5",
        12: "total_tanpa_sepeda_motor",
        13: "total_dengan_sepeda_motor",
    }
}

config_data_profil = [
    [6, 5, "catatan"],
    [3, 9, "jam_puncak_arah_1"],
    [4, 9, "vol_jam_puncak_arah_1"],
    [6, 9, "jam_puncak_arah_2"],
    [7, 9, "vol_jam_puncak_arah_2"],
]
data = []


with open_workbook("/mnt/d/PROJ UI/REV-4 (onedrive-FIN)/REV-4 (onedrive-FIN)/JABO/JBO-22.xlsb") as wb:
    for sheet_name in wb.sheets:
        if str(sheet_name).startswith(("HK_Arah-1")):
            with wb.get_sheet(sheet_name) as sheet:
                base_dict = {}
                base_dict["filename"] = "filename"
                for i, row in enumerate(sheet):
                    for c in range(len(row)):
                        if row[c].v:
                            for config in config_data:
                                if i == config[0] and c == config[1]:
                                    base_dict[config[2]] = int(row[c].v) if isinstance(row[c].v, float) else row[c].v
                
                d_dict = {}
                for i, row in enumerate(sheet):
                    for c in range(len(row)):
                        if i >= table_data["i"][0] and i <= table_data["i"][1] and c >= table_data["c"][0] and c <= table_data["c"][1]:
                            if c == 1:
                                print(row[c].v)
                                d_dict[table_data["mapping_col"][c]] = row[c].v if row[c].v else None
                                # print(table_data["mapping_col"][c], row[c].v if row[c].v else None)
                            else:
                                print(table_data["mapping_col"][c], i, row[c].v)
                                d_dict[table_data["mapping_col"][c]] = int(row[c].v) if isinstance(row[c].v, float) else 0
                                # print(table_data["mapping_col"][c], int(row[c].v) if row[c].v else None)
                    if d_dict:
                        complete_dict = {**base_dict, **d_dict}
                        data.append(complete_dict)

# df = pl.DataFrame(data)
# # df["rentang_survei"].unique()
# # df
             
# ddl = generate_postgres_ddl(df=df, table_name="survey_traffic_counting")    
# print(ddl)                  

In [134]:
dd = {}
if dd:
    print("OK")

# Survei Travel Journey

In [ ]:
wb = openpyxl.load_workbook(survey_travel_journey_xlsx, data_only=True)  # data_only=False keeps the formulas

columns = [
    'timestamp',
    'kmph',
    'y',
    'x',
    'waktu',
    'jarak',
    'waktu_menit',
    'jarak_km',
]

In [ ]:
data = []
for config in sheet_config_travel_journey:
    for sheet_name in config["sheets"]:
        
        sheet_name = wb[sheet_name]
        for table in config["tables"]:
            # Convert column letters to column numbers
            start_col = column_index_from_string(table["start_col"])
            end_col = column_index_from_string(table["end_col"])
            print(table["row"], table["start_col"], table["end_col"])

            
            for row in sheet_name.iter_rows(min_row=table["row"], min_col=start_col, max_col=end_col, values_only=True):
                d_dict = {}
                if all(cell is None for cell in row):
                    break
                
                for i in range(0, len(columns)):
                    d_dict[columns[i]] = row[i] if not isinstance(row[i], datetime.time) else row[i].strftime("%H:%M:%S")
                d_dict["arah"] = table["name"]
                d_dict["sheet"] = sheet_name
                data.append(d_dict)
            print(sheet_name, len(data))

In [ ]:
df_survey_travel_journey = pl.DataFrame(data)
print(df_survey_travel_journey)

ddl_survey_travel_journey = generate_postgres_ddl(df=df_survey_travel_journey, table_name="survey_travel_journey")
print(ddl_survey_travel_journey)
df_to_db(df=df_survey_travel_journey, table_name="survey_travel_journey")

In [2]:
import folium
folium.__version__

'0.17.0'

In [3]:
m = folium.Map(location=(30, 20), zoom_start=4)

def style_function(feature):
    area = int(feature['properties']['area'])
    return {
        'fillOpacity': 0.5,
        'weight': 0.5,
        'fillColor': 'green' if area < 10008018378 \
               else 'orange'if area< 20008018378\
                else 'red'
    }

gjson = folium.GeoJson("https://raw.githubusercontent.com/stefanocudini/leaflet-geojson-selector/master/examples/italy-regions.json", 
                       style_function=style_function
                      ).add_to(m)

In [42]:
from utils.util import engine, read_database
query = """
    select
        kode_lokasi,
        sheet,
        FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)) as start_rentang_survei,
        (FLOOR(cast(trim(split_part(rentang_survei, '-', 1)) as float)):: varchar) || '.00 - ' || (CEIL(cast(trim(split_part(rentang_survei, '-', 2)) as float)):: varchar) || '.00' as rentang_survei_,
        sum("Gol-6") as gol_6,
        sum("Gol-1-a") as gol_1_a,
        sum("Gol-1-b") as gol_1_b,
        sum("Gol-1-c") as gol_1_c,
        sum("Gol-1-d") as gol_1_d,
        sum("Gol-1-a") as gol_1_e,
        sum("Gol-2") as gol_2,
        sum("Gol-3") as gol_3,
        sum("Gol-4") as gol_4,
        sum("Gol-5") as gol_5,
        sum(total_tanpa_sepeda_motor) as total_tanpa_sepeda_motor,
        sum(total_dengan_sepeda_motor) as total_dengan_sepeda_motor
    from
        survey_traffic_counting
    where kode_lokasi = 'ADDJBO-1'
    group by
        kode_lokasi,
        sheet,
        start_rentang_survei,
        rentang_survei_
"""
df = read_database(engine=engine, query=query)
df

,kode_lokasi,sheet,start_rentang_survei,rentang_survei_,gol_6,gol_1_a,gol_1_b,gol_1_c,gol_1_d,gol_1_e,gol_2,gol_3,gol_4,gol_5,total_tanpa_sepeda_motor,total_dengan_sepeda_motor
0,ADDJBO-1,HK_Arah-1,0.0,0.00 - 1.00,174,43,0,0,0,43,7,42,6,2,110,284
1,ADDJBO-1,HK_Arah-1,1.0,1.00 - 2.00,78,17,0,0,0,17,13,43,3,0,84,162
2,ADDJBO-1,HK_Arah-1,2.0,2.00 - 3.00,63,20,1,0,0,20,9,67,3,4,113,176
3,ADDJBO-1,HK_Arah-1,3.0,3.00 - 4.00,14,17,0,0,1,17,12,63,6,1,106,120
4,ADDJBO-1,HK_Arah-1,4.0,4.00 - 5.00,33,10,0,0,1,10,13,55,10,0,95,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,ADDJBO-1,HL_Arah-2,19.0,19.00 - 20.00,910,163,4,1,0,163,15,3,3,1,200,1110
92,ADDJBO-1,HL_Arah-2,20.0,20.00 - 21.00,789,150,2,1,0,150,15,20,5,1,204,993
93,ADDJBO-1,HL_Arah-2,21.0,21.00 - 22.00,742,127,1,0,4,127,11,14,7,3,177,919
94,ADDJBO-1,HL_Arah-2,22.0,22.00 - 23.00,524,111,1,0,1,111,9,34,1,4,169,693


In [44]:
df_stats_hk_jabo = df.loc[(df['kode_lokasi'] == 'ADDJBO-1') & (df['sheet'].str.startswith("HK"))]
# df_stats_hk_jabo
df_stats_hk_jabo = df_stats_hk_jabo.groupby('rentang_survei_', as_index=False).sum()
df_stats_hk_jabo = df_stats_hk_jabo.sort_values(by='start_rentang_survei').drop(columns=['start_rentang_survei', 'kode_lokasi', 'sheet'])
df_stats_hk_jabo

,rentang_survei_,gol_6,gol_1_a,gol_1_b,gol_1_c,gol_1_d,gol_1_e,gol_2,gol_3,gol_4,gol_5,total_tanpa_sepeda_motor,total_dengan_sepeda_motor
0,0.00 - 1.00,290,66,0,0,3,66,13,182,15,6,298,588
1,1.00 - 2.00,129,32,0,0,0,32,15,165,7,2,231,360
12,2.00 - 3.00,99,35,1,0,0,35,15,133,5,4,206,305
17,3.00 - 4.00,39,25,1,0,1,25,15,117,11,1,181,220
18,4.00 - 5.00,89,35,0,0,2,35,19,92,18,0,175,264
19,5.00 - 6.00,504,119,12,4,6,119,25,48,16,1,243,747
20,6.00 - 7.00,2935,316,53,2,1,316,19,7,6,2,422,3357
21,7.00 - 8.00,3012,299,48,1,1,299,23,8,1,0,413,3425
22,8.00 - 9.00,2070,286,34,0,1,286,68,25,8,4,486,2556
23,9.00 - 10.00,1680,275,28,1,0,275,109,33,13,6,556,2236


In [46]:
a = [1 for _ in range(3)]
a

[1, 1, 1]